In [1]:
import pandas as pd
import numpy as np
import torch
import urllib.request
from padelpy import from_smiles
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import sys
sys.path.insert(0, '/home/mm22d016/TransPolymer')
from transformers import AdamW, get_linear_schedule_with_warmup, RobertaModel, RobertaConfig, RobertaTokenizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from PolymerSmilesTokenization import PolymerSmilesTokenizer
from dataset import Downstream_Dataset, DataAugmentation
from sklearn.model_selection import train_test_split
from copy import deepcopy
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors    
#Chem.Descriptors.CalcMolDescriptors(mol, missingVal=None, silent=True)
#Chem.MolFromSmiles('CC')

In [3]:
dataset = pd.read_csv('../data/new_feat_Li_group.csv')
dataset

,Polymer,SMILES descriptor 1,SMILES descriptor 2,Anion,Anion Smiles,log Li:functional group,Comonomer percentage,approxMW(kDa),approxTg,chain architecture,...,Comonomer 2 RDF100m,Comonomer 2 SMR_VSA7,Comonomer 2 SlogP_VSA10,Comonomer 2 SlogP_VSA12,Comonomer 2 SlogP_VSA6,Comonomer 2 TDB10s,Comonomer 2 VSA_EState10,Comonomer 2 fr_aryl_methyl,drying vacuum,logCond60
0,polyethylene carbonate,*CCOC(=O)O*,*CCO*,TFSI,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-1.125997,95.20,37.0,-3.0,S_1,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0,2,-5.720000
1,polyethylene carbonate,*CCOC(=O)O*,*CCO*,TFSI,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-0.700029,95.20,37.0,-20.0,S_1,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0,2,-5.190000
2,polyethylene carbonate,*CCOC(=O)O*,*CCO*,TFSI,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-0.347846,95.20,37.0,-33.0,S_1,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0,2,-4.810000
3,polyethylene carbonate,*CCOC(=O)O*,*CCO*,TFSI,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,0.078123,95.20,37.0,-62.0,S_1,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0,2,-4.250000
4,polyethylene carbonate,*CCOC(=O)O*,*CCO*,N(SO2C2F5)2,C(F)(F)(F)C(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)...,-1.255939,95.20,37.0,-14.0,S_1,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0,2,-5.560000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,poly (n-butyl glycidyl ether-alt-carbon dioxide),CCCCOCC(C*)O*,*OC(*)=O,TFSI,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-1.301030,50.00,190.0,-20.0,S_2,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0,2,-5.836820
267,poly (methoxy ethyl glycidyl ether-alt-carbon ...,COCCOCC(C*)O*,*OC(*)=O,TFSI,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-1.477121,50.00,17.0,-55.0,S_2,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0,2,-4.528742
268,poly (ethylene oxide-co-methyl glycidyl ether),*CCO*,*CCCOCO*,CF3SO3,C(F)(F)(F)S(=O)(=O)[O-],-1.204120,78.30,1000.0,-68.6,S_2,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0,2,-4.771455
269,poly (ethylene oxide-co-ethoxyethyl glycidyl e...,*CCO*,*CCOCCOCOCC*,CF3SO3,C(F)(F)(F)S(=O)(=O)[O-],-1.204120,82.94,1000.0,-53.0,S_2,...,0.240164,0.0,0.0,0.0,0.0,15.620742,0.0,0,2,-4.613009


In [4]:
ds = rdMolDescriptors.MQNs_(m)
Descriptors.CalcMolDescriptors(mol, missingVal=None, silent=True)

In [5]:
unique_polymers = dataset['Polymer'].unique()
unique_anions = dataset['Anion'].unique()

In [6]:
j=1
groups = np.zeros(len(dataset['Anion']))
lookup = {}
for polym in unique_polymers:
    for anion in unique_anions:
        lookup[(polym,anion)] = j
        j+=1
        
for ind in dataset.index:
    groups[ind] = lookup[(dataset['Polymer'][ind],dataset['Anion'][ind])]

In [7]:
from sklearn.model_selection import GroupShuffleSplit
gss = GroupShuffleSplit(n_splits=1,train_size=0.8,random_state=1)
train_inds,test_inds = next(gss.split(dataset.iloc[:,:-1],dataset.iloc[:,-1], groups))
train_data = dataset.iloc[train_inds]
test_data = dataset.iloc[test_inds]

In [8]:
train_data

,Polymer,SMILES descriptor 1,SMILES descriptor 2,Anion,Anion Smiles,log Li:functional group,Comonomer percentage,approxMW(kDa),approxTg,chain architecture,...,Comonomer 2 RDF100m,Comonomer 2 SMR_VSA7,Comonomer 2 SlogP_VSA10,Comonomer 2 SlogP_VSA12,Comonomer 2 SlogP_VSA6,Comonomer 2 TDB10s,Comonomer 2 VSA_EState10,Comonomer 2 fr_aryl_methyl,drying vacuum,logCond60
0,polyethylene carbonate,*CCOC(=O)O*,*CCO*,TFSI,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-1.125997,95.20,37.0,-3.0,S_1,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0,2,-5.720000
1,polyethylene carbonate,*CCOC(=O)O*,*CCO*,TFSI,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-0.700029,95.20,37.0,-20.0,S_1,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0,2,-5.190000
2,polyethylene carbonate,*CCOC(=O)O*,*CCO*,TFSI,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-0.347846,95.20,37.0,-33.0,S_1,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0,2,-4.810000
3,polyethylene carbonate,*CCOC(=O)O*,*CCO*,TFSI,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,0.078123,95.20,37.0,-62.0,S_1,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0,2,-4.250000
4,polyethylene carbonate,*CCOC(=O)O*,*CCO*,N(SO2C2F5)2,C(F)(F)(F)C(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)...,-1.255939,95.20,37.0,-14.0,S_1,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0,2,-5.560000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,poly (n-butyl glycidyl ether-alt-carbon dioxide),CCCCOCC(C*)O*,*OC(*)=O,TFSI,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-1.301030,50.00,190.0,-20.0,S_2,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0,2,-5.836820
267,poly (methoxy ethyl glycidyl ether-alt-carbon ...,COCCOCC(C*)O*,*OC(*)=O,TFSI,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-1.477121,50.00,17.0,-55.0,S_2,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0,2,-4.528742
268,poly (ethylene oxide-co-methyl glycidyl ether),*CCO*,*CCCOCO*,CF3SO3,C(F)(F)(F)S(=O)(=O)[O-],-1.204120,78.30,1000.0,-68.6,S_2,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0,2,-4.771455
269,poly (ethylene oxide-co-ethoxyethyl glycidyl e...,*CCO*,*CCOCCOCOCC*,CF3SO3,C(F)(F)(F)S(=O)(=O)[O-],-1.204120,82.94,1000.0,-53.0,S_2,...,0.240164,0.0,0.0,0.0,0.0,15.620742,0.0,0,2,-4.613009


In [9]:
test_data

,Polymer,SMILES descriptor 1,SMILES descriptor 2,Anion,Anion Smiles,log Li:functional group,Comonomer percentage,approxMW(kDa),approxTg,chain architecture,...,Comonomer 2 RDF100m,Comonomer 2 SMR_VSA7,Comonomer 2 SlogP_VSA10,Comonomer 2 SlogP_VSA12,Comonomer 2 SlogP_VSA6,Comonomer 2 TDB10s,Comonomer 2 VSA_EState10,Comonomer 2 fr_aryl_methyl,drying vacuum,logCond60
8,polyethylene carbonate,*CCOC(=O)O*,*CCO*,BF4,[B-](F)(F)(F)F,-0.758140,95.2,37.0000,-23.000000,S_1,...,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0,2,-6.510000
9,polyethylene carbonate,*CCOC(=O)O*,*CCO*,BF4,[B-](F)(F)(F)F,-0.448155,95.2,37.0000,-29.000000,S_1,...,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0,2,-5.240000
10,polyethylene carbonate,*CCOC(=O)O*,*CCO*,BF4,[B-](F)(F)(F)F,-0.269119,95.2,37.0000,-47.000000,S_1,...,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0,2,-5.080000
11,polyethylene carbonate,*CCOC(=O)O*,*CCO*,BF4,[B-](F)(F)(F)F,-0.135672,95.2,37.0000,-69.000000,S_1,...,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0,2,-4.340000
100,Poly (ethylene succinate),*OC(=O)CCC(=O)O*,*OC(=O)CCC(=O)O*,SCN,N#C[S-],-1.397940,100.0,23.4000,-3.000000,S_1,...,3.770000e-13,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0,1,-7.693178
113,polyvinyl alcohol,OC(*)C*,*CCOC(=O)C*,TFSI,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-0.602060,98.0,125000.0000,58.000000,S_1,...,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0,2,-6.403910
116,polyvinyl alcohol,OC(*)C*,*CCOC(=O)C*,TFSI,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-0.602060,98.0,125000.0000,59.000000,S_1,...,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0,0,-7.116898
121,poly(N-methylethylenimine),CN(C*)C*,CN(C*)C*,CF3SO3,C(F)(F)(F)S(=O)(=O)[O-],-1.204120,100.0,162707.0000,-42.071376,S_1,...,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0,2,-6.017039
122,poly(N-methylethylenimine),CN(C*)C*,CN(C*)C*,CF3SO3,C(F)(F)(F)S(=O)(=O)[O-],-1.000000,100.0,162707.0000,-33.793230,S_1,...,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0,2,-5.971895
123,poly(N-methylethylenimine),CN(C*)C*,CN(C*)C*,CF3SO3,C(F)(F)(F)S(=O)(=O)[O-],-0.770852,100.0,162707.0000,4.194260,S_1,...,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0,2,-7.030151


In [10]:
train_data.drop(columns=['Polymer'],inplace=True)

/tmp/ipykernel_81213/1729097393.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.drop(columns=['Polymer'],inplace=True)


In [11]:
test_data.drop(columns=['Polymer'],inplace=True)

/tmp/ipykernel_81213/3992807619.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.drop(columns=['Polymer'],inplace=True)


In [13]:
train_data.reset_index(inplace=True,drop=True)

In [14]:
test_data.reset_index(inplace=True,drop=True)

In [15]:
train_data

,SMILES descriptor 1,SMILES descriptor 2,Anion,Anion Smiles,log Li:functional group,Comonomer percentage,approxMW(kDa),approxTg,chain architecture,Comonomer 1 CIC1,...,Comonomer 2 RDF100m,Comonomer 2 SMR_VSA7,Comonomer 2 SlogP_VSA10,Comonomer 2 SlogP_VSA12,Comonomer 2 SlogP_VSA6,Comonomer 2 TDB10s,Comonomer 2 VSA_EState10,Comonomer 2 fr_aryl_methyl,drying vacuum,logCond60
0,*CCOC(=O)O*,*CCO*,TFSI,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-1.125997,95.20,37.0,-3.0,S_1,1.625815,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0,2,-5.720000
1,*CCOC(=O)O*,*CCO*,TFSI,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-0.700029,95.20,37.0,-20.0,S_1,1.625815,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0,2,-5.190000
2,*CCOC(=O)O*,*CCO*,TFSI,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-0.347846,95.20,37.0,-33.0,S_1,1.625815,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0,2,-4.810000
3,*CCOC(=O)O*,*CCO*,TFSI,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,0.078123,95.20,37.0,-62.0,S_1,1.625815,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0,2,-4.250000
4,*CCOC(=O)O*,*CCO*,N(SO2C2F5)2,C(F)(F)(F)C(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)...,-1.255939,95.20,37.0,-14.0,S_1,1.625815,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0,2,-5.560000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,CCCCOCC(C*)O*,*OC(*)=O,TFSI,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-1.301030,50.00,190.0,-20.0,S_2,2.910195,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0,2,-5.836820
210,COCCOCC(C*)O*,*OC(*)=O,TFSI,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-1.477121,50.00,17.0,-55.0,S_2,2.959037,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0,2,-4.528742
211,*CCO*,*CCCOCO*,CF3SO3,C(F)(F)(F)S(=O)(=O)[O-],-1.204120,78.30,1000.0,-68.6,S_2,1.945531,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0,2,-4.771455
212,*CCO*,*CCOCCOCOCC*,CF3SO3,C(F)(F)(F)S(=O)(=O)[O-],-1.204120,82.94,1000.0,-53.0,S_2,1.945531,...,0.240164,0.0,0.0,0.0,0.0,15.620742,0.0,0,2,-4.613009


In [16]:
test_data

,SMILES descriptor 1,SMILES descriptor 2,Anion,Anion Smiles,log Li:functional group,Comonomer percentage,approxMW(kDa),approxTg,chain architecture,Comonomer 1 CIC1,...,Comonomer 2 RDF100m,Comonomer 2 SMR_VSA7,Comonomer 2 SlogP_VSA10,Comonomer 2 SlogP_VSA12,Comonomer 2 SlogP_VSA6,Comonomer 2 TDB10s,Comonomer 2 VSA_EState10,Comonomer 2 fr_aryl_methyl,drying vacuum,logCond60
0,*CCOC(=O)O*,*CCO*,BF4,[B-](F)(F)(F)F,-0.758140,95.2,37.0000,-23.000000,S_1,1.625815,...,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0,2,-6.510000
1,*CCOC(=O)O*,*CCO*,BF4,[B-](F)(F)(F)F,-0.448155,95.2,37.0000,-29.000000,S_1,1.625815,...,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0,2,-5.240000
2,*CCOC(=O)O*,*CCO*,BF4,[B-](F)(F)(F)F,-0.269119,95.2,37.0000,-47.000000,S_1,1.625815,...,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0,2,-5.080000
3,*CCOC(=O)O*,*CCO*,BF4,[B-](F)(F)(F)F,-0.135672,95.2,37.0000,-69.000000,S_1,1.625815,...,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0,2,-4.340000
4,*OC(=O)CCC(=O)O*,*OC(=O)CCC(=O)O*,SCN,N#C[S-],-1.397940,100.0,23.4000,-3.000000,S_1,2.160964,...,3.770000e-13,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0,1,-7.693178
5,OC(*)C*,*CCOC(=O)C*,TFSI,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-0.602060,98.0,125000.0000,58.000000,S_1,1.289960,...,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0,2,-6.403910
6,OC(*)C*,*CCOC(=O)C*,TFSI,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,-0.602060,98.0,125000.0000,59.000000,S_1,1.289960,...,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0,0,-7.116898
7,CN(C*)C*,CN(C*)C*,CF3SO3,C(F)(F)(F)S(=O)(=O)[O-],-1.204120,100.0,162707.0000,-42.071376,S_1,2.560324,...,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0,2,-6.017039
8,CN(C*)C*,CN(C*)C*,CF3SO3,C(F)(F)(F)S(=O)(=O)[O-],-1.000000,100.0,162707.0000,-33.793230,S_1,2.560324,...,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0,2,-5.971895
9,CN(C*)C*,CN(C*)C*,CF3SO3,C(F)(F)(F)S(=O)(=O)[O-],-0.770852,100.0,162707.0000,4.194260,S_1,2.560324,...,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0,2,-7.030151
